In [2]:
from __future__ import print_function

import sys

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import sklearn


import random

In [3]:
import sklearn.datasets

mnist = sklearn.datasets.mldata.fetch_mldata('MNIST original')
X, Y = mnist['data'], np.array(mnist['target'], dtype='int')

ntrain_dev = 60000
X_train_dev, Y_train_dev = X[:ntrain_dev], Y[:ntrain_dev]
X_test, Y_test = X[ntrain_dev:], Y[ntrain_dev:]

permutation = np.random.permutation(ntrain_dev)
ntrain = ntrain_dev * 90 // 100

X_train, Y_train = X_train_dev[permutation][:ntrain], Y_train_dev[permutation][:ntrain]
X_dev, Y_dev = X_train_dev[permutation][ntrain:], Y_train_dev[permutation][ntrain:]

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


# Prepare data

In [96]:
def filter_by_digits(x, y, digits, limit=None):
    indices = [i for i in range(len(x)) if y[i] in digits]
    limit = limit or len(indices)
    return x[indices][:limit], y[indices][:limit]

In [97]:
# Normalize to [-1; -1]
def normalize(x, y):
    x = (x * 2 / 255.0) - 1.0
    return x, y
    
x_train_dev, y_train_dev = normalize(x_train_dev, y_train_dev)
x_train, y_train = normalize(x_train, y_train)
x_dev, y_dev = normalize(x_dev, y_dev)
x_test, y_test = normalize(x_test, y_test)

In [101]:
def prepare_toy_sets():
    x_train_dev, y_train_dev = filter_by_digits(X_train_dev, Y_train_dev, cool_digits, 3000)
    x_train, y_train = filter_by_digits(X_train, Y_train, cool_digits, 3000)
    x_dev, y_dev = filter_by_digits(X_dev, Y_dev, cool_digits, 300)
    x_test, y_test = filter_by_digits(X_test, Y_test, cool_digits, 300)
    
    x_train_dev, y_train_dev = normalize(x_train_dev, y_train_dev)
    x_train, y_train = normalize(x_train, y_train)
    x_dev, y_dev = normalize(x_dev, y_dev)
    x_test, y_test = normalize(x_test, y_test)
    
def prepare_full_sets():
    x_train_dev, y_train_dev = X_train_dev, Y_train_dev
    x_train, y_train = X_train, Y_train
    x_dev, y_dev = X_dev, Y_dev
    x_test, y_test = X_test, Y_test
    
    x_train_dev, y_train_dev = normalize(x_train_dev, y_train_dev)
    x_train, y_train = normalize(x_train, y_train)
    x_dev, y_dev = normalize(x_dev, y_dev)
    x_test, y_test = normalize(x_test, y_test)

# Визуализация данных

In [20]:
def visualize(x, title=None):
    x = np.array(x, dtype='uint8').reshape((28,28))
    if title:
        plt.title(title)
    plt.imshow(x, cmap='gray')
    plt.show()

In [93]:
nvisualize = 3
for i in range(nvisualize):
    num = random.randint(0, len(x_train) - 1)
    print(x_train[num])
    #visualize(x_train[num], title='This is {}'.format(y_train[num]))

3000


# SVM

In [94]:
import sklearn.svm
import sklearn.model_selection

prepare_toy_sets()

tuned_parameters = [
    {
        'kernel': ['rbf'], 
        'gamma': [0.001, 0.005, 0.01],
        'C': [0.25, 0.5, 1, 2, 4, 8]
    },
]

grid_search = sklearn.model_selection.GridSearchCV(
    sklearn.svm.SVC(),
    tuned_parameters,
    cv=10,
    n_jobs=4
)
grid_search.fit(x_train, y_train)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

0.769 (+/-0.035) for {'C': 0.25, 'gamma': 0.001, 'kernel': 'rbf'}
0.891 (+/-0.022) for {'C': 0.25, 'gamma': 0.005, 'kernel': 'rbf'}
0.906 (+/-0.028) for {'C': 0.25, 'gamma': 0.01, 'kernel': 'rbf'}
0.851 (+/-0.036) for {'C': 0.5, 'gamma': 0.001, 'kernel': 'rbf'}
0.906 (+/-0.031) for {'C': 0.5, 'gamma': 0.005, 'kernel': 'rbf'}
0.924 (+/-0.028) for {'C': 0.5, 'gamma': 0.01, 'kernel': 'rbf'}
0.884 (+/-0.026) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.922 (+/-0.030) for {'C': 1, 'gamma': 0.005, 'kernel': 'rbf'}
0.936 (+/-0.027) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.900 (+/-0.026) for {'C': 2, 'gamma': 0.001, 'kernel': 'rbf'}
0.929 (+/-0.023) for {'C': 2, 'gamma': 0.005, 'kernel': 'rbf'}
0.939 (+/-0.018) for {'C': 2, 'gamma': 0.01, 'kernel': 'rbf'}
0.914 (+/-0.030) for {'C': 4, 'gamma': 0.001, 'kernel': 'rbf'}
0.933 (+/-0.028) for {'C': 4, 'gamma': 0.005, 'kernel': 'rbf'}
0.942 (+/-0.026) for {'C': 4, 'gamma': 0.01, 'kernel': 'rbf'}
0.922 (+/-0.027) for {'C': 8, 'gamma': 0.001,

In [98]:
from sklearn.svm import SVC
from sklearn import metrics

prepare_full_sets()

model = SVC(kernel="rbf", C=4, gamma=.001)
model.fit(x_train_dev, y_train_dev)

predicted = model.predict(x_test)
print("Accuracy: %0.4f" % metrics.accuracy_score(y_test, predicted))

out = model.score(x_test, y_test)
print("out: %s" % out)

Accuracy: 0.9449
out: 0.9449


In [104]:
import sklearn.svm
import sklearn.model_selection

prepare_toy_sets()

tuned_parameters = [
    {
        'kernel': ['rbf'], 
        'gamma': [0.005, 0.075, 0.01],
        'C': np.arange(2, 8.5, 0.5)
    },
]

grid_search = sklearn.model_selection.GridSearchCV(
    sklearn.svm.SVC(),
    tuned_parameters,
    cv=10,
    n_jobs=4
)
grid_search.fit(x_train, y_train)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

0.929 (+/-0.023) for {'C': 2.0, 'gamma': 0.005, 'kernel': 'rbf'}
0.880 (+/-0.036) for {'C': 2.0, 'gamma': 0.075, 'kernel': 'rbf'}
0.939 (+/-0.018) for {'C': 2.0, 'gamma': 0.01, 'kernel': 'rbf'}
0.931 (+/-0.023) for {'C': 2.5, 'gamma': 0.005, 'kernel': 'rbf'}
0.880 (+/-0.036) for {'C': 2.5, 'gamma': 0.075, 'kernel': 'rbf'}
0.940 (+/-0.022) for {'C': 2.5, 'gamma': 0.01, 'kernel': 'rbf'}
0.934 (+/-0.024) for {'C': 3.0, 'gamma': 0.005, 'kernel': 'rbf'}
0.880 (+/-0.036) for {'C': 3.0, 'gamma': 0.075, 'kernel': 'rbf'}
0.941 (+/-0.023) for {'C': 3.0, 'gamma': 0.01, 'kernel': 'rbf'}
0.933 (+/-0.025) for {'C': 3.5, 'gamma': 0.005, 'kernel': 'rbf'}
0.880 (+/-0.036) for {'C': 3.5, 'gamma': 0.075, 'kernel': 'rbf'}
0.941 (+/-0.024) for {'C': 3.5, 'gamma': 0.01, 'kernel': 'rbf'}
0.933 (+/-0.028) for {'C': 4.0, 'gamma': 0.005, 'kernel': 'rbf'}
0.880 (+/-0.036) for {'C': 4.0, 'gamma': 0.075, 'kernel': 'rbf'}
0.942 (+/-0.026) for {'C': 4.0, 'gamma': 0.01, 'kernel': 'rbf'}
0.934 (+/-0.026) for {'C': 4.5

In [111]:
from sklearn.svm import SVC
from sklearn import metrics

prepare_full_sets()

model = SVC(kernel="rbf", C=2.7, gamma=.0075)
model.fit(x_train_dev, y_train_dev)

predicted = model.predict(x_test)
print("Test: %0.3f" % metrics.accuracy_score(y_test, predicted))

Test: 0.975
